# Next

- [ ] aggregate process metrics 
    - once it goes to multi-GPU it gets weird - i think this is why half the runs don't work, the other half do
    - including assinging the unique ID
    - decide which metrics to sum, which to avg

- [ ] log metrics both at (1) the individual per prompt, (2) the per-batch level, and (3) the experiment level (e.g. number of input/output tokens, energy etc etc)
- [ ] add Carbon Tracker

# Low priority
- [ ] add logic for task-specific metrics in the reporting
- [ ] how is GPU / CPU power calculated by CodeCarbon?
- [ ] sort out the print statements in experiments to supress (i) the code carbon output, (ii) the final JSON print

# Blockers

- [ ] vLLM -> CUDA runs out of memory
- [ ] Optimum benchmarking library -> CUDA runs out of memory

# Done
- [x] calculate computation metrics
- [x] put the text_gen_inference() into the helper functionn (as it is pretty generic)
- [x] us Optimum library 
- [x] sort out distributed inference using accelerate / vLLM. At the moment it's not working on either


1) need to not count each run (4*5 = 20, when there's just 5)
2) need to append and not overwrite

In [1]:
from datetime import datetime

current_time = datetime.now().strftime("%B %d, %Y at %I:%M:%S %p")
print(current_time)


March 04, 2025 at 06:34:54 PM
